# Imports

In [ ]:
import math as math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.preprocessing import StandardScaler, Normalizer, KBinsDiscretizer, FunctionTransformer, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Data loading

In [ ]:
data_cols = ["RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe"]
class_col = "class"
data_cols_names = {
    "RI": "współczynnik załamania",
    "Na": "sód",
    "Mg": "magnez",
    "Al": "glin",
    "Si": "krzem",
    "K": "potas",
    "Ca": "wapń",
    "Ba": "bar",
    "Fe": "żelazo"
    }
col_names = ["id", *data_cols, class_col]

file_path = "./glass.data"
dataset = pd.read_csv(file_path, names=col_names)

# Data exploration

In [ ]:
dataset.shape

In [ ]:
dataset.head(10)

In [ ]:
dataset.info()

In [ ]:
dataset.loc[:, data_cols].describe()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset[class_col].value_counts().sort_index()

In [ ]:
color_palette = matplotlib.colormaps["Dark2"]

class_values = dataset[class_col].value_counts()
plt.bar(class_values.index, class_values.values, color=color_palette(range(len(class_values))))
plt.ylabel('Liczba wystąpień')
plt.xlabel('Klasa szkła')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))

for i, column in enumerate(data_cols):
    ax = axes[i // 3, i % 3]
    dataset.boxplot(column=[column], ax=ax)
    ax.set_title(f'{data_cols_names[column].capitalize()}')
    ax.set_ylabel('Wartość')

plt.tight_layout()
plt.show()

In [ ]:
for i in range(1, 8):
    print(f"Glass class: {i}")
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))

    for j, column in enumerate(data_cols):
        ax = axes[j // 3, j % 3]
        dataset[dataset[class_col] == i].boxplot(column=[column], ax=ax)
        ax.set_title(f'{data_cols_names[column].capitalize()}')
        ax.set_ylabel('Wartość')

    plt.tight_layout()
    plt.show()

In [ ]:
sns.pairplot(dataset.loc[:, dataset.columns != 'id'], hue="class", palette="hls")

In [ ]:
sns.heatmap(dataset[data_cols].corr(), annot=True, fmt=".2f", vmin=-1.0, vmax=1.0, cmap="viridis")

# Preprocessing

In [ ]:
fixed_rng = np.random.RandomState(0)
Y = dataset[class_col]
X_full = dataset[data_cols]
X_missing = X_full.mask(fixed_rng.random(X_full.shape) < 0.05)

X_full_train, X_full_test, Y_full_train, Y_full_test = train_test_split(X_full, Y, test_size=0.3, stratify=Y, random_state=fixed_rng)
X_missing_train, X_missing_test, Y_missing_train, Y_missing_test = train_test_split(X_missing, Y, test_size=0.3, stratify=Y, random_state=fixed_rng)

In [ ]:
Y_full_test.value_counts().sort_index()

In [ ]:
Y_missing_test.value_counts().sort_index()

In [ ]:
imputers = {
    "KNN-1" : KNNImputer(n_neighbors=1),
    "KNN-2" : KNNImputer(n_neighbors=2),
    "MEAN" : SimpleImputer(strategy="mean"),
    "MEDIAN" : SimpleImputer(strategy="median"),
    "MOST-FQ" : SimpleImputer(strategy="most_frequent")
}

transformers = {
    "NONE" : "passthrough",
    "NORMALIZE" : Normalizer(),
    "STANDARIZE" : StandardScaler(),
    "DISCRETIZE-2" : KBinsDiscretizer(n_bins=2, strategy="quantile", encode="onehot-dense"),
    "DISCRETIZE-3" : KBinsDiscretizer(n_bins=3, strategy="quantile", encode="onehot-dense"),
    "DISCRETIZE-5" : KBinsDiscretizer(n_bins=5, strategy="quantile", encode="onehot-dense"),
    "SELECTION-2" : SelectKBest(k=2),
    "SELECTION-3" : SelectKBest(k=3),
    "SELECTION-5" : SelectKBest(k=5),
    "PCA" : PCA(random_state=fixed_rng)
}

In [ ]:
sns.FacetGrid(
    pd.concat(
        pd.DataFrame(
            make_pipeline(transformers[n])
                .fit_transform(dataset[["Si", "Al"]].to_numpy(), dataset[class_col]),
            columns=["Si", "Al"]
        ).assign(**{"transformer": n, "class": dataset[class_col]})
        for n in ["NONE", "NORMALIZE", "STANDARIZE", "PCA"]
    ), col="transformer", col_wrap=2, sharex=False, sharey=False
).map_dataframe(sns.scatterplot, x="Si", y="Al", hue="class", palette="hls")

# Classification

In [ ]:
classifiers = {
    "NAIVE-1e-3": GaussianNB(var_smoothing=1e-3),
    "NAIVE-1e-5": GaussianNB(var_smoothing=1e-5),
    "NAIVE-1e-8": GaussianNB(var_smoothing=1e-8),
    "DECISION-TREE-3": DecisionTreeClassifier(max_depth=3, criterion="entropy", random_state=fixed_rng),
    "DECISION-TREE-5": DecisionTreeClassifier(max_depth=5, criterion="entropy", random_state=fixed_rng),
    "DECISION-TREE-8": DecisionTreeClassifier(max_depth=8, criterion="entropy", random_state=fixed_rng),
    "SVC": LinearSVC(random_state=fixed_rng),
    "RANDOM-FOREST": RandomForestClassifier(random_state=fixed_rng)
}

In [ ]:
search_pipeline_missing = Pipeline([
    ("imputer", "passthrough"),
    ("transformer", "passthrough"),
    ("classifier", next(iter(classifiers.values())))
])

search_pipeline_full = Pipeline([
    ("transformer", "passthrough"),
    ("classifier", next(iter(classifiers.values())))
])

In [ ]:
grid_search_missing = GridSearchCV(search_pipeline_missing, {
    "imputer": list(imputers.values()),
    "transformer": list(transformers.values()),
    "classifier": list(classifiers.values())
}, n_jobs=-1)

grid_search_missing.fit(X_missing_train, Y_missing_train)

print(grid_search_missing.best_score_)
best_pipeline = grid_search_missing.best_estimator_
best_pipeline

In [ ]:
grid_search_full = GridSearchCV(search_pipeline_full, {
    "transformer": list(transformers.values()),
    "classifier": list(classifiers.values())
}, n_jobs=-1)

grid_search_full.fit(X_full_train, Y_full_train)

print(grid_search_full.best_score_)
best_pipeline = grid_search_full.best_estimator_
best_pipeline

# Results

In [ ]:
def lookup(what, where):
    match where:
        case "imputer":
            d = imputers
        case "transformer":
            d = transformers
        case "classifier":
            d = classifiers
    return next(k for k in d if d[k] == what)

polska_gurom = {"classifier": "Klasyfikator", "transformer": "Przetwarzanie danych", "imputer": "Uzupełnianie danych"}

In [ ]:

res = grid_search_missing.cv_results_
for i, p in enumerate(grid_search_missing.best_params_.keys()):
    x, y, std = zip(*sorted([
        max([
            (lookup(param, p), entry[1], entry[2])
            for entry in zip(res["params"], res["mean_test_score"], res["std_test_score"])
            if entry[0][p] == param
        ], key=lambda x: x[1])
        for param in grid_search_missing.param_grid[p]
    ], key=lambda x: x[1]))

    ax = plt.axes()
    bar_val = ax.barh(x, y, xerr=std)
    ax.bar_label(bar_val, label_type='center')
    ax.set_title(polska_gurom[p])
    ax.set_xlabel("Dokładność")
    plt.show()

In [ ]:

res = grid_search_full.cv_results_
for i, p in enumerate(grid_search_full.best_params_.keys()):
    x, y, std = zip(*sorted([
        max([
            (lookup(param, p), entry[1], entry[2])
            for entry in zip(res["params"], res["mean_test_score"], res["std_test_score"])
            if entry[0][p] == param
        ], key=lambda x: x[1])
        for param in grid_search_full.param_grid[p]
    ], key=lambda x: x[1]))

    ax = plt.axes()
    bar_val = ax.barh(x, y, xerr=std)
    ax.bar_label(bar_val, label_type='center')
    ax.set_title(polska_gurom[p])
    ax.set_xlabel("Dokładność")
    plt.show()

In [ ]:
forest_grid = GridSearchCV(
    make_pipeline(
        "passthrough",
        RandomForestClassifier(random_state=fixed_rng)
    ),
    {
        "randomforestclassifier__n_estimators": np.arange(1, 150, 5)
    },
    n_jobs=-1
)

forest_grid.fit(X_full_train, Y_full_train)

In [ ]:
res = sns.lineplot(data=pd.DataFrame({
    "n_estimators": forest_grid.cv_results_["param_randomforestclassifier__n_estimators"],
    "score": forest_grid.cv_results_["mean_test_score"]
}), x="n_estimators", y="score", errorbar="se", )

res.set(xlabel="Liczba estymatorów", ylabel="Dokładność")

In [ ]:
grid = GridSearchCV(
    make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        "passthrough",
        RandomForestClassifier(random_state=fixed_rng)
    ),
    {},
    n_jobs=-1
)

grid.fit(X_full_train, Y_full_train)
estimator = grid.best_estimator_

In [ ]:
estimator.fit(X_full_train, Y_full_train)
predictions = estimator.predict(X_full_test)
print(accuracy_score(Y_full_test, predictions))
print(classification_report(Y_full_test, predictions))

In [ ]:
estimator.fit(X_missing_train, Y_missing_train)
predictions = estimator.predict(X_missing_test)
print(accuracy_score(Y_missing_test, predictions))
print(classification_report(Y_missing_test, predictions))